In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [102]:
########################################
# DataFrame Setup Function
########################################

def setup_dataframes(dataset_name, infer_mode, features, attribute=None):
    if attribute:
        log_path = dataset_name + '/monitoring_' + attribute + '_quality_log.csv'
        train = 'train_batch'
        infer = 'infer_batch'
    else:
        log_path = dataset_name + '/monitoring_quality_log.csv'
        train = 'skip_training_starting_batch'
        infer = 'batch'

    df_full = pd.read_csv(log_path, sep=';')
#     df_full = df_full[['infer_mode', 'features', 'dk_cells', train,
#                        infer, 'precision', 'recall', 'f1', ]]
    df_full.drop_duplicates(keep='last', inplace=True)

    df = df_full.loc[(df_full['infer_mode'] == infer_mode) & (df_full['features'] == features)]

    df_list = []
    for skip_training_starting_batch in df[train].unique().tolist():
        tmp_df = df.loc[df[train] == skip_training_starting_batch]
        tmp_df = tmp_df.astype({'dk_cells': 'int32', infer: 'int32', 'total_errors': 'int32',
                                'correct_repairs': 'int32', 'total_repairs_grdt': 'int32',
                                'precision': 'float64', 'recall': 'float64', 'f1': 'float64'})

        df_list.append(tmp_df)

    return df_list

In [179]:
########################################
# Chart Plotting Function
########################################

def plot_charts(df_list, attribute=None):
    if attribute:
        train = 'train_batch'
        infer = 'infer_batch'
    else:
        train = 'skip_training_starting_batch'
        infer = 'batch'

    count = 0

    chart_rows = 33
    chart_columns = 3
    fig, axes = plt.subplots(chart_rows, chart_columns, sharex=True, sharey=True,
                             figsize=(chart_columns * 7, chart_rows * 5))

    for i in range(chart_rows):
        for j in range(chart_columns):
            current_df = df_list[count]
            count += 1
            
            current_df.loc[current_df['dk_cells'] == 0, ['precision', 'recall', 'f1']] = 1

            axes[i, j].plot(infer, 'precision',
                            data=current_df, marker='', color=palette(1), label='precision', linewidth=3)
            axes[i, j].plot(infer, 'recall',
                            data=current_df, marker='', color=palette(2), label='recall', linewidth=3)
            axes[i, j].plot(infer, 'f1',
                            data=current_df, marker='', color=palette(3), label='f1', linewidth=3)
            axes[i, j].legend(loc="lower right")

            skip_training_starting_batch = current_df[train].iloc[0]
            title_str = 'Training up to batch {}'.format(skip_training_starting_batch)
            axes[i, j].set(title=title_str)

            if j == 0:
                axes[i, j].set(ylabel='Percentage')
            if i == chart_rows - 1:
                axes[i, j].set(xlabel='Batch')

    fig.tight_layout()

In [131]:
def plot_charts_acc(df_list, attribute=None):
    if attribute:
        train = 'train_batch'
        infer = 'infer_batch'
    else:
        train = 'skip_training_starting_batch'
        infer = 'batch'

    count = 0
    prev_values = {'total_errors':  0, 'correct_repairs': 0, 'total_repairs_grdt': 0}

    chart_rows = 5  # 33
    chart_columns = 3
    fig, axes = plt.subplots(chart_rows, chart_columns, sharex=True, sharey=True,
                             figsize=(chart_columns * 7, chart_rows * 5))

    for i in range(chart_rows):
        for j in range(chart_columns):
            current_df = df_list[count]
            count += 1

            for metric in prev_values.keys():
                # Sets the first entry to the accumulated value up to it.
                current_df.loc[current_df.index[0], metric] = current_df.loc[current_df.index[0], metric] + prev_values[metric]
                prev_values[metric] = current_df.loc[current_df.index[0], metric]
                # Gets the accumulated values.
                current_df[metric] = current_df[metric].cumsum()

            current_df['acc_precision'] = current_df['correct_repairs'] / current_df['total_repairs_grdt']
            current_df['acc_recall'] = current_df['correct_repairs'] / current_df['total_errors']
            current_df['acc_f1'] = (2 * current_df['acc_precision'] * current_df['acc_recall']) \
                / (current_df['acc_precision'] + current_df['acc_recall'])
            
            axes[i, j].plot(infer, 'acc_precision',
                            data=current_df, marker='', color=palette(1), label='Precision', linewidth=3)
            axes[i, j].plot(infer, 'acc_recall',
                            data=current_df, marker='', color=palette(2), label='Recall', linewidth=3)
            axes[i, j].plot(infer, 'acc_f1',
                            data=current_df, marker='', color=palette(3), label='F1', linewidth=3)
            axes[i, j].legend(loc="lower right")

            skip_training_starting_batch = current_df[train].iloc[0]
            title_str = 'Training up to batch {}'.format(skip_training_starting_batch)
            axes[i, j].set(title=title_str)

            if j == 0:
                axes[i, j].set(ylabel='Percentage')
            if i == chart_rows - 1:
                axes[i, j].set(xlabel='Batch')

    fig.tight_layout()

In [132]:
########################################
# Charts: Setup
########################################

plt.rcParams.update({'font.size': 18})
plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('tab10')

In [184]:
########################################
# Charts: Analysis
########################################

# An attribute name or None.
attribute = None

# 1st argument: 'dk' or 'all'.
# 2nd argument: 'incremental' or 'global'.

plot_charts_acc(setup_dataframes('hospital', 'dk', 'incremental', attribute), attribute)

In [162]:
dataset_name = 'hospital'
df = pd.read_csv(dataset_name + '/monitoring_quality_log.csv', sep=';')

df

infer_mode     features  train_using_all_batches  \
0              dk       global                     True   
1              dk       global                     True   
2              dk       global                     True   
3              dk       global                     True   
4              dk       global                     True   
5              dk       global                     True   
6              dk       global                     True   
7              dk       global                     True   
8              dk       global                     True   
9              dk       global                     True   
10             dk       global                     True   
11             dk       global                     True   
12             dk       global                     True   
13             dk       global                     True   
14             dk       global                     True   
15             dk       global                     True   
16             dk       global                     True   
17             dk       global                     True   
18             dk       global                     True   
19             dk       global                     True   
20             dk       global                     True   
21             dk       global                     True   
22             dk       global                     True   
23             dk       global                     True   
24             dk       global                     True   
25             dk       global                     True   
26             dk       global                     True   
27             dk       global                     True   
28             dk       global                     True   
29             dk       global                     True   
...           ...          ...                      ...   
20569  infer_mode     features  train_using_all_batches   
20570          dk  incremental                     True   
20571          dk  incremental                     True   
20572          dk  incremental                     True   
20573  infer_mode     features  train_using_all_batches   
20574         all  incremental                     True   
20575         all  incremental                     True   
20576         all  incremental                     True   
20577         all  incremental                     True   
20578  infer_mode     features  train_using_all_batches   
20579         all       global                     True   
20580         all       global                     True   
20581         all       global                     True   
20582  infer_mode     features  train_using_all_batches   
20583          dk  incremental                     True   
20584          dk  incremental                     True   
20585  infer_mode     features  train_using_all_batches   
20586         all  incremental                     True   
20587         all  incremental                     True   
20588         all  incremental                     True   
20589  infer_mode     features  train_using_all_batches   
20590         all       global                     True   
20591         all       global                     True   
20592  infer_mode     features  train_using_all_batches   
20593          dk  incremental                     True   
20594         all  incremental                     True   
20595         all  incremental                     True   
20596  infer_mode     features  train_using_all_batches   
20597         all       global                     True   
20598         all  incremental                     True   

       skip_training_starting_batch  batch  dk_cells  training_cells  \
0                                 1      1         3             154   
1                                 1      2         4             154   
2                                 1      3         7             154   
3                                 1      4         3        